In [1]:
from pyspark import SparkConf, SparkContext

conf = SparkConf().setMaster("local").setAppName("FriendsByAge")
sc = SparkContext(conf = conf)

22/05/16 00:10:01 WARN Utils: Your hostname, Gleui-MacBookPro.local resolves to a loopback address: 127.0.0.1; using 172.30.1.18 instead (on interface en0)
22/05/16 00:10:01 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/05/16 00:10:02 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
filepath ='/Users/glebang/SparkCourse/data/fakefriends.csv'

In [3]:
lines = sc.textFile(f"file:///{filepath}")
lines.collect()[:3]

['0,Will,33,385', '1,Jean-Luc,26,2', '2,Hugh,55,221']

In [12]:
def parse(lines):
    fields= lines.split(',')
    age = int(fields[2])
    count = int(fields[3])
    return age, count

In [13]:
ageFriendsRDD = lines.map(parse)
ageFriendsRDD.collect()[:5]

[(33, 385), (26, 2), (55, 221), (40, 465), (68, 21)]

In [14]:
totalFriends = ageFriendsRDD.reduceByKey(lambda x, y : x + y)
totalFriends.collect()[:3]

[(33, 3904), (26, 4115), (55, 3842)]

In [15]:
countFriends = ageFriendsRDD.mapValues(lambda x : (x, 1))
countFriends.collect()[:5]

[(33, (385, 1)), (26, (2, 1)), (55, (221, 1)), (40, (465, 1)), (68, (21, 1))]

In [16]:
totalsByAge = countFriends.reduceByKey(lambda x, y : (x[0] + y[0], x[1] + y[1]))
totalsByAge.collect()[:5]

[(33, (3904, 12)),
 (26, (4115, 17)),
 (55, (3842, 13)),
 (40, (4264, 17)),
 (68, (2696, 10))]

In [18]:
avgFriends = totalsByAge.mapValues(lambda x : x[0] / x[1])
avgFriends.collect()[:3]

[(33, 325.3333333333333), (26, 242.05882352941177), (55, 295.53846153846155)]

In [19]:
sc.stop()